# Load dataset

## Import the pandas library and perform data reading for analysis.

In [63]:
import pandas as pd
train = pd.read_csv('train.csv', sep='\t', header=None)
test = pd.read_csv('test.csv', sep='\t', header=None)

## Adding column names to data.

In [64]:
train.columns = ["text",'labels']
test.columns = ["text",'labels']

## The first 10 texts of the training set are read in the format "text, labels". The labels may contain multiple sentiment categories, each sentiment is separated by ','.

In [65]:
train.head(10)

,text,labels
0,My favourite food is anything I didn't have to...,27
1,"Now if he does off himself, everyone will thin...",27
2,WHY THE FUCK IS BAYLESS ISOING,2
3,To make her feel threatened,14
4,Dirty Southern Wankers,3
5,OmG pEyToN iSn'T gOoD eNoUgH tO hElP uS iN tHe...,26
6,Yes I heard abt the f bombs! That has to be wh...,15
7,We need more boards and to create a bit more s...,"8,20"
8,Damn youtube and outrage drama is super lucrat...,0
9,It might be linked to the trust factor of your...,27


In [66]:
test.head(10)

,text,labels
0,I’m really sorry about your situation :( Altho...,25
1,It's wonderful because it's awful. At not with.,0
2,"Kings fan here, good luck to you guys! Will be...",13
3,"I didn't know that, thank you for teaching me ...",15
4,They got bored from haunting earth for thousan...,27
5,Thank you for asking questions and recognizing...,15
6,You’re welcome,15
7,100%! Congrats on your job too!,15
8,I’m sorry to hear that friend :(. It’s for the...,24
9,"Girlfriend weak as well, that jump was pathetic.",25


# Import related libraries.

In [67]:
import os
import paddle
import paddlenlp

# Data pre-processing

## For the 28 micro-sentiment multi-label classification scenario, where a sentence may correspond to multiple sentiment category labels, the sentiment labels of the dataset need to be transformed using One-Hot coding first, with "0" indicating absence and "1" indicating presence for each sentiment.

### Create sentiment label mapping relationships.

In [68]:
label_vocab = {
    0: "admiration",
    1: "amusement",
    2: "anger",
    3: "annoyance",
    4: "approval",
    5: "caring",
    6: "confusion",
    7: "curiosity",
    8: "desire",
    9: "disappointment",
    10: "disapproval",
    11: "disgust",
    12: "embarrassment",
    13: "excitement",
    14: "fear",
    15: "gratitude",
    16: "grief",
    17: "joy",
    18: "love",
    19: "nervousness",
    20: "optimism",
    21: "pride",
    22: "realization",
    23: "relief",
    24: "remorse",
    25: "sadness",
    26: "surprise",
    27: "neutral"
}

### Customize the dataset, read the data file, create the dataset and define the data type as MapDataset.

In [69]:
import re

from paddlenlp.datasets import load_dataset

# Clear invalid characters
def clean_text(text):
    text = text.replace("\r", "").replace("\n", "")
    text = re.sub(r"\\n\n", ".", text)
    return text

# Define the read data set function
def read_custom_data(filepath, is_one_hot=True):
    f = open(filepath)
    while True:
        line = f.readline()
        if not line:
            break
        data = line.strip().split('\t')
        # One-hot processing for 28 types of micro sentiment tags
        if is_one_hot:
            labels = [float(1) if str(i) in data[1].split(',') else float(0) for i in range(28)]  # 28 types
        else:
            labels = [int(d) for d in data[1].split(',')]
        yield {"text": clean_text(data[0]), "labels": labels}
    f.close()

In [70]:
# load_dataset() to Create dataset.
# lazy=False，The dataset is returned as a MapDataset type.
# Pre-processing of training and validation sets.
train_ds = load_dataset(read_custom_data, filepath='train.csv', lazy=False) 
test_ds = load_dataset(read_custom_data, filepath='test.csv', lazy=False)

### Print dataset.

In [71]:
print("datatype:", type(train_ds))
print("training dataset example:", train_ds[0])
print("testing dataset example:", test_ds[0])

datatype: <class 'paddlenlp.datasets.dataset.MapDataset'>
training dataset example: {'text': "My favourite food is anything I didn't have to cook myself.", 'labels': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]}
testing dataset example: {'text': 'I’m really sorry about your situation :( Although I love the names Sapphira, Cirilla, and Scarlett!', 'labels': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0]}


## Load Chinese ERNIE 3.0 pre-training model and word splitter

In [72]:
from paddlenlp.transformers import AutoModelForSequenceClassification, AutoTokenizer

model_name = "ernie-3.0-medium-zh"   # ERNIE3.0 model
num_classes = 28  # 28 classification mission
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_classes=num_classes)
tokenizer = AutoTokenizer.from_pretrained(model_name)

[2023-04-22 16:42:15,075] [    INFO] - We are using <class 'paddlenlp.transformers.ernie.modeling.ErnieForSequenceClassification'> to load 'ernie-3.0-medium-zh'.
[2023-04-22 16:42:15,082] [    INFO] - Model config ErnieConfig {
  "attention_probs_dropout_prob": 0.1,
  "enable_recompute": false,
  "fuse": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23": "LABEL_23",
    "24": "LABEL_24",
    "25": "LABEL_25",
    "26": "LABEL_26",
    "27": "LABEL_27"


## Process the raw data into a model-acceptable format.

In [73]:
import functools
import numpy as np

from paddle.io import DataLoader, BatchSampler
from paddlenlp.data import DataCollatorWithPadding

# Data pre-processing function to convert text into integer sequences using a word splitter.
def preprocess_function(examples, tokenizer, max_seq_length):
    result = tokenizer(text=examples["text"], max_seq_len=max_seq_length)
    result["labels"] = examples["labels"]
    return result

trans_func = functools.partial(preprocess_function, tokenizer=tokenizer, max_seq_length=64)
train_ds = train_ds.map(trans_func)
test_ds = test_ds.map(trans_func)

# function is constructed to extend the different length sequences to the maximum length of the data in the batch, and then stack the data.
collate_fn = DataCollatorWithPadding(tokenizer)

# Define the BatchSampler, select the batch size and whether to randomly jumble the DataLoader.
train_batch_sampler = BatchSampler(train_ds, batch_size=32, shuffle=True)
test_batch_sampler = BatchSampler(test_ds, batch_size=16, shuffle=False)
train_data_loader = DataLoader(dataset=train_ds, batch_sampler=train_batch_sampler, collate_fn=collate_fn)
test_data_loader = DataLoader(dataset=test_ds, batch_sampler=test_batch_sampler, collate_fn=collate_fn)

## Define model validation metrics.

In [74]:
import numpy as np
import sklearn
from sklearn.metrics import roc_auc_score, f1_score, precision_score, recall_score
from paddle.metric import Metric

# Customize MultiLabelReport evaluation metrics.
class MultiLabelReport(Metric):
    """
    AUC and F1 Score for multi-label text classification task.
    """

    def __init__(self, name='MultiLabelReport', average='micro'):
        super(MultiLabelReport, self).__init__()
        self.average = average
        self._name = name
        self.reset()

    def f1_score(self, y_prob):
        '''
        Returns the f1 score by searching the best threshhold
        '''
        best_score = 0
        for threshold in [i * 0.01 for i in range(100)]:
            self.y_pred = y_prob > threshold
            score = sklearn.metrics.f1_score(y_pred=self.y_pred, y_true=self.y_true, average=self.average)
            if score > best_score:
                best_score = score
                precison = precision_score(y_pred=self.y_pred, y_true=self.y_true, average=self.average)
                recall = recall_score(y_pred=self.y_pred, y_true=self.y_true, average=self.average)
        return best_score, precison, recall

    def reset(self):
        """
        Resets all of the metric state.
        """
        self.y_prob = None
        self.y_true = None

    def update(self, probs, labels):
        if self.y_prob is not None:
            self.y_prob = np.append(self.y_prob, probs.numpy(), axis=0)
        else:
            self.y_prob = probs.numpy()
        if self.y_true is not None:
            self.y_true = np.append(self.y_true, labels.numpy(), axis=0)
        else:
            self.y_true = labels.numpy()

    def accumulate(self):
        auc = roc_auc_score(
            y_score=self.y_prob, y_true=self.y_true, average=self.average)
        f1_score, precison, recall = self.f1_score(y_prob=self.y_prob)
        return auc, f1_score, precison, recall

    def name(self):
        """
        Returns metric name
        """
        return self._name

# Building the training model.

## Select an optimization strategy and run configuration.

In [75]:
import time
import paddle.nn.functional as F

# AdamW optimizer, cross-entropy loss function, custom MultiLabelReport evaluation metrics.
optimizer = paddle.optimizer.AdamW(learning_rate=6e-2, parameters=model.parameters(), weight_decay=0.01)
criterion = paddle.nn.MSELoss()
metric = MultiLabelReport()

## Model training and validation.

In [76]:
import paddle
import numpy as np
import paddle.nn.functional as F

# Build the validation set evaluate function.
@paddle.no_grad()
def evaluate(model, criterion, metric, data_loader, label_vocab, if_return_results=True):
    model.eval()
    metric.reset()
    losses = []
    results = []
    for batch in data_loader:
        input_ids, token_type_ids, labels = batch['input_ids'], batch['token_type_ids'], batch['labels']
        logits = model(input_ids, token_type_ids)
        loss = criterion(logits, labels)
        probs = F.sigmoid(logits)
        losses.append(loss.numpy())
        metric.update(probs, labels)
        if if_return_results:
            probs = probs.tolist()
            for prob in probs:
                result = []
                for c, pred in enumerate(prob):
                    if pred > 0.5:
                        result.append(label_vocab[c])
                results.append(','.join(result))

    auc, f1_score, precison, recall = metric.accumulate()
    print("eval loss: %.5f, auc: %.5f, f1 score: %.5f, precison: %.5f, recall: %.5f" %
          (np.mean(losses), auc, f1_score, precison, recall))
    model.train()
    metric.reset()
    if if_return_results:
        return results
    else:
        return f1_score

In [77]:
epochs = 3 # training times
ckpt_dir = "ernie_ckpt" # Folder for saving model parameters during training

global_step = 0  # Number of iterations
tic_train = time.time()
best_f1_score = 0

# Model Training
for epoch in range(1, epochs + 1):
    for step, batch in enumerate(train_data_loader, start=1):
        input_ids, token_type_ids, labels = batch['input_ids'], batch['token_type_ids'], batch['labels']

        # Calculate model output, loss function value, classification probability value, accuracy, f1 score.
        logits = model(input_ids, token_type_ids)
        loss = criterion(logits, labels)
        probs = F.sigmoid(logits)
        metric.update(probs, labels)
        auc, f1_score, _, _ = metric.accumulate()

        # Print the loss function value, accuracy, f1 score, and computation speed for each 10 iterations.
        global_step += 1
        if global_step % 10 == 0:
            print(
                "global step %d, epoch: %d, batch: %d, loss: %.5f, auc: %.5f, f1 score: %.5f, speed: %.2f step/s"
                % (global_step, epoch, step, loss, auc, f1_score,
                    10 / (time.time() - tic_train)))
            tic_train = time.time()
        
        # Reverse gradient passback with updated parameters.
        loss.backward()
        optimizer.step()
        optimizer.clear_grad()
        
        # Every 40 iterations, evaluate the current trained model, save the current best model parameters and word list of the word splitter, etc.
        if global_step % 40 == 0:
            save_dir = ckpt_dir
            if not os.path.exists(save_dir):
                os.makedirs(save_dir)
            eval_f1_score = evaluate(model, criterion, metric, test_data_loader, label_vocab, if_return_results=False)
            if eval_f1_score > best_f1_score:
                best_f1_score = eval_f1_score
                model.save_pretrained(save_dir)
                tokenizer.save_pretrained(save_dir)

global step 10, epoch: 1, batch: 10, loss: 156.88049, auc: 0.53645, f1 score: 0.09235, speed: 0.26 step/s
global step 20, epoch: 1, batch: 20, loss: 97.19123, auc: 0.50663, f1 score: 0.08330, speed: 0.24 step/s
global step 30, epoch: 1, batch: 30, loss: 78.28904, auc: 0.50238, f1 score: 0.08199, speed: 0.23 step/s
global step 40, epoch: 1, batch: 40, loss: 14.44917, auc: 0.50678, f1 score: 0.08175, speed: 0.25 step/s


[2023-04-22 16:48:51,025] [    INFO] - Configuration saved in ernie_ckpt/config.json


eval loss: 20.94166, auc: 0.41441, f1 score: 0.07997, precison: 0.04165, recall: 1.00000


[2023-04-22 16:48:51,296] [    INFO] - tokenizer config file saved in ernie_ckpt/tokenizer_config.json
[2023-04-22 16:48:51,297] [    INFO] - Special tokens file saved in ernie_ckpt/special_tokens_map.json


global step 50, epoch: 1, batch: 50, loss: 3.16498, auc: 0.50748, f1 score: 0.08550, speed: 0.04 step/s
global step 60, epoch: 1, batch: 60, loss: 1.36867, auc: 0.50643, f1 score: 0.08181, speed: 0.25 step/s
global step 70, epoch: 1, batch: 70, loss: 1.11207, auc: 0.50599, f1 score: 0.08093, speed: 0.24 step/s
global step 80, epoch: 1, batch: 80, loss: 1.34504, auc: 0.50857, f1 score: 0.08204, speed: 0.23 step/s


...

global step 2010, epoch: 2, batch: 483, loss: 0.50941, auc: 0.51589, f1 score: 0.11404, speed: 0.04 step/s
global step 2020, epoch: 2, batch: 493, loss: 0.47023, auc: 0.52253, f1 score: 0.10340, speed: 0.24 step/s
global step 2030, epoch: 2, batch: 503, loss: 0.30310, auc: 0.52142, f1 score: 0.09496, speed: 0.23 step/s
global step 2040, epoch: 2, batch: 513, loss: 0.86430, auc: 0.52154, f1 score: 0.09262, speed: 0.22 step/s
eval loss: 0.70352, auc: 0.63356, f1 score: 0.18575, precison: 0.11996, recall: 0.41144
global step 2050, epoch: 2, batch: 523, loss: 0.61276, auc: 0.54455, f1 score: 0.12252, speed: 0.04 step/s
global step 2060, epoch: 2, batch: 533, loss: 0.98375, auc: 0.53985, f1 score: 0.09895, speed: 0.24 step/s
global step 2070, epoch: 2, batch: 543, loss: 1.52096, auc: 0.53368, f1 score: 0.09091, speed: 0.23 step/s
global step 2080, epoch: 2, batch: 553, loss: 1.11054, auc: 0.53974, f1 score: 0.09192, speed: 0.23 step/s
eval loss: 0.97088, auc: 0.60601, f1 score: 0.13664, pre

global step 2650, epoch: 2, batch: 1123, loss: 1.19134, auc: 0.52524, f1 score: 0.08446, speed: 0.04 step/s
global step 2660, epoch: 2, batch: 1133, loss: 1.70851, auc: 0.52113, f1 score: 0.08373, speed: 0.24 step/s
global step 2670, epoch: 2, batch: 1143, loss: 2.78616, auc: 0.51916, f1 score: 0.08286, speed: 0.23 step/s
global step 2680, epoch: 2, batch: 1153, loss: 1.71282, auc: 0.52103, f1 score: 0.08274, speed: 0.23 step/s
eval loss: 1.62872, auc: 0.63935, f1 score: 0.24524, precison: 0.19412, recall: 0.33291
global step 2690, epoch: 2, batch: 1163, loss: 1.10228, auc: 0.53747, f1 score: 0.11033, speed: 0.04 step/s
global step 2700, epoch: 2, batch: 1173, loss: 1.60682, auc: 0.54570, f1 score: 0.09668, speed: 0.22 step/s
global step 2710, epoch: 2, batch: 1183, loss: 2.12933, auc: 0.52549, f1 score: 0.09024, speed: 0.23 step/s
global step 2720, epoch: 2, batch: 1193, loss: 2.42481, auc: 0.51527, f1 score: 0.09195, speed: 0.23 step/s
eval loss: 2.03013, auc: 0.43712, f1 score: 0.09

eval loss: 2.80680, auc: 0.40755, f1 score: 0.08454, precison: 0.04451, recall: 0.83963
global step 3290, epoch: 3, batch: 236, loss: 3.54658, auc: 0.50789, f1 score: 0.08074, speed: 0.04 step/s
global step 3300, epoch: 3, batch: 246, loss: 3.49094, auc: 0.52078, f1 score: 0.08503, speed: 0.23 step/s
global step 3310, epoch: 3, batch: 256, loss: 5.06637, auc: 0.51686, f1 score: 0.08404, speed: 0.23 step/s
global step 3320, epoch: 3, batch: 266, loss: 2.38631, auc: 0.51222, f1 score: 0.08421, speed: 0.22 step/s
eval loss: 2.05143, auc: 0.61151, f1 score: 0.30401, precison: 0.32928, recall: 0.28235
global step 3330, epoch: 3, batch: 276, loss: 3.59837, auc: 0.48816, f1 score: 0.08136, speed: 0.04 step/s
global step 3340, epoch: 3, batch: 286, loss: 2.90734, auc: 0.50742, f1 score: 0.08503, speed: 0.24 step/s
global step 3350, epoch: 3, batch: 296, loss: 2.61356, auc: 0.51136, f1 score: 0.08426, speed: 0.22 step/s
global step 3360, epoch: 3, batch: 306, loss: 3.66315, auc: 0.51281, f1 sco

eval loss: 1.03034, auc: 0.40461, f1 score: 0.08486, precison: 0.04441, recall: 0.95197
global step 3930, epoch: 3, batch: 876, loss: 1.41608, auc: 0.53589, f1 score: 0.09394, speed: 0.04 step/s
global step 3940, epoch: 3, batch: 886, loss: 1.63146, auc: 0.52347, f1 score: 0.08722, speed: 0.23 step/s
global step 3950, epoch: 3, batch: 896, loss: 1.65955, auc: 0.52016, f1 score: 0.08767, speed: 0.23 step/s
global step 3960, epoch: 3, batch: 906, loss: 0.96283, auc: 0.52294, f1 score: 0.09498, speed: 0.24 step/s
eval loss: 0.80610, auc: 0.44414, f1 score: 0.08603, precison: 0.04511, recall: 0.92827
global step 3970, epoch: 3, batch: 916, loss: 1.14845, auc: 0.54123, f1 score: 0.11380, speed: 0.04 step/s
global step 3980, epoch: 3, batch: 926, loss: 1.02579, auc: 0.53040, f1 score: 0.09757, speed: 0.25 step/s
global step 3990, epoch: 3, batch: 936, loss: 1.28595, auc: 0.53888, f1 score: 0.09412, speed: 0.23 step/s
global step 4000, epoch: 3, batch: 946, loss: 1.58919, auc: 0.53299, f1 sco

global step 4560, epoch: 3, batch: 1506, loss: 1.16999, auc: 0.51840, f1 score: 0.08765, speed: 0.22 step/s
eval loss: 1.19964, auc: 0.52657, f1 score: 0.17231, precison: 0.11965, recall: 0.30779
global step 4570, epoch: 3, batch: 1516, loss: 1.86271, auc: 0.55032, f1 score: 0.10816, speed: 0.04 step/s
global step 4580, epoch: 3, batch: 1526, loss: 1.92682, auc: 0.54441, f1 score: 0.09834, speed: 0.23 step/s


## Load trained model

In [78]:
# Load the optimal parameters of the trained model.
model.set_dict(paddle.load('ernie_ckpt/model_state.pdparams'))

# Load the parameters of the previously trained model.
# model.set_dict(paddle.load('/home/aistudio/work/model_state.pdparams'))

# Model Validation.
print("ERNIE 3.0 performance on GoEmotions micro-emotion 28 classification test set：", end= " ")
results = evaluate(model, criterion, metric, test_data_loader, label_vocab)

ERNIE 3.0 performance on GoEmotions micro-emotion 28 classification test set： eval loss: 0.35317, auc: 0.67353, f1 score: 0.30401, precison: 0.32928, recall: 0.28235


# Use the model to make predictions about the sentiment contained in the sentences.

In [79]:
# Define data loading and processing functions.
from paddlenlp.data import JiebaTokenizer, Pad, Stack, Tuple, Vocab
def convert_example(example, tokenizer, max_seq_length=64, is_test=False):
    qtconcat = example["text"]
    encoded_inputs = tokenizer(text=qtconcat, max_seq_len=max_seq_length)
    input_ids = encoded_inputs["input_ids"]
    token_type_ids = encoded_inputs["token_type_ids"]
    if not is_test:
        label = np.array([example["label"]], dtype="int64")
        return input_ids, token_type_ids, label
    else:
        return input_ids, token_type_ids

# Define the model prediction function.
def predict(model, data, tokenizer, label_vocab, batch_size=1, max_seq=64):
    examples = []
    # Process input data (list format) into a format acceptable to the model.
    for text in data:
        input_ids, segment_ids = convert_example(
            text,
            tokenizer,
            max_seq_length=max_seq,
            is_test=True)
        examples.append((input_ids, segment_ids))

    batchify_fn = lambda samples, fn=Tuple(
        Pad(axis=0, pad_val=tokenizer.pad_token_id),  # input id
        Pad(axis=0, pad_val=tokenizer.pad_token_id),  # segment id
    ): fn(samples)

    # Seperates data into some batches.
    batches = []
    one_batch = []
    for example in examples:
        one_batch.append(example)
        if len(one_batch) == batch_size:
            batches.append(one_batch)
            one_batch = []
    if one_batch:
        # The last batch whose size is less than the config batch_size setting.
        batches.append(one_batch)

    results = []
    model.eval()
    for batch in batches:
        input_ids, segment_ids = batchify_fn(batch)
        input_ids = paddle.to_tensor(input_ids)
        segment_ids = paddle.to_tensor(segment_ids)
        logits = model(input_ids, segment_ids)
        probs = F.sigmoid(logits)
        probs = probs.tolist()
        # The results were processed by selecting the sentiment categories with probability greater than 0.5.
        for prob in probs:
            result = []
            for c, pred in enumerate(prob):
                if pred > 0.5:
                    result.append(label_vocab[c])
            results.append(','.join(result))
    return results  # Return prediction results

In [80]:
# Define the text data to be subjected to micro-sentiment analysis.
data = [
    # 0 admiration
    {"text": 'You do a great job!'},
    # 1 amusement
    {"text": 'Lets have fun!'},
    # 2 anger
    {"text":"You shut your mouth"},
    # 3 annoyance
    {"text": 'You are so annoyed'},
    # 4 approval
    {"text": 'You are allowed to do this'},
    # 5 caring & # 7 Curiosity
    {"text": 'Are you feeling well?'},
    # 6 confusion
    {"text": 'This problem is so hard and I cannot solve this problem'},
    # 7 curiosity
    {"text":'Why would I do that?'},
    # 8 desire
    {"text": 'I want this gift so much'},
    # 9 disappointment
    {"text": 'I am very disappointed by everything you have done to me'},
    # 10 disapproval
    {"text": 'You are not admitted to the college.'},
    # 11 disgust
    {"text": 'Thats absolutely disgusting.'},
    # 12 embarrassment
    {"text": 'Thats so embarrassing.'},
    # 13 excitement
    {"text": 'I am so excited'},
    # 14 fear
    {"text": 'I am so scared of skydiving'},
    # 15 gratitude
    {"text":"Thank you."},
    # 16 grief
    {"text": 'My grandpa passed away'},
    # 17 joy
    {"text": 'Happy Birthday'},
    # 18 love
    {"text": 'I love you so much'},
    # 19 nervousness
    {"text": 'I am so nervous.'},
    # 20 neutral
    {"text": 'It is just so so.'},
    # 21 optimism
    {"text": 'Successful people only focus on giving their best effort.'},
    # 22 pride
    {"text": 'I am so proud of you.'},
    # 23 realization
    {"text": 'Thank you for letting me realizing this rule.'},
    # 24 relief 
    {"text": 'You are doing better than you think you are'},
    # 25 remorse
    {"text": 'I am guilty.'},
    # 26 sadness
    {"text": 'I am so sad.'},
    # 27 surprise
    {"text": 'I am so surprised that you made it.'},
]

# Model Predictions.
labels =  predict(model, data, tokenizer, label_vocab, batch_size=1)

# Output prediction results
for idx, text in enumerate(data):
    print('Text: {} \t Labels: {}'.format(text['text'], labels[idx]))

Text: You do a great job! 	 Labels: admiration,amusement,approval,confusion,disapproval,disgust,excitement,fear,joy,nervousness,optimism,neutral
Text: Lets have fun! 	 Labels: admiration,amusement,approval,confusion,disapproval,disgust,excitement,fear,joy,nervousness,optimism,neutral
Text: You shut your mouth 	 Labels: admiration,amusement,approval,confusion,disapproval,disgust,excitement,fear,joy,nervousness,optimism,neutral
Text: You are so annoyed 	 Labels: admiration,amusement,approval,confusion,disapproval,disgust,excitement,fear,joy,nervousness,optimism,neutral
Text: You are allowed to do this 	 Labels: admiration,amusement,approval,confusion,disapproval,disgust,excitement,fear,joy,nervousness,optimism,neutral
Text: Are you feeling well? 	 Labels: admiration,amusement,approval,confusion,disapproval,disgust,excitement,fear,joy,nervousness,optimism,neutral
Text: This problem is so hard and I cannot solve this problem 	 Labels: admiration,amusement,approval,confusion,disapproval,dis